In [1]:
import sys
sys.path.append( '/anaconda/envs/py37_pytorch/lib/python3.7')
sys.path.append('/anaconda/envs/py37_pytorch/lib/python3.7/site-packages')

In [2]:
%load_ext autoreload
%autoreload 2

import glob
import logging
import numpy as np
from importlib import reload  # Not needed in Python 2
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.models import ResNet
from torch.utils.data import DataLoader,Dataset


from dataset.ASdataset import AS_Data
from dataset.ASdataset_obs_train_input import AS_Data_obs

device = torch.device("cuda"  if torch.cuda.is_available() else "cpu")
reload(logging)
logging.basicConfig(level=logging.INFO,#控制台打印的日志级别
                    filename='logging.txt',
                    filemode='a',##模式，有w和a，w就是写模式，每次都会重新写日志，覆盖之前的日志
                    #a是追加模式，默认如果不写的话，就是追加模式
                    format=
                    '%(asctime)s : %(message)s',
                    )

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [8]:
# GET ALL DATA
data_list = []
for filename in sorted(glob.glob('/AS_data/METCRO2D_npy/METCRO2D_2015*')):
    print(filename)
    data = np.load(filename)
    print(data.shape)
    data_list.append(data)
all_data = np.concatenate(data_list,axis=0)

/AS_data/METCRO2D_npy/METCRO2D_2015_01_744_34_182_232.npy
(744, 34, 182, 232)
/AS_data/METCRO2D_npy/METCRO2D_2015_04_720_34_182_232.npy
(720, 34, 182, 232)
/AS_data/METCRO2D_npy/METCRO2D_2015_07_744_34_182_232.npy
(744, 34, 182, 232)
/AS_data/METCRO2D_npy/METCRO2D_2015_10_744_34_182_232.npy
(744, 34, 182, 232)


In [9]:
norm_data = []
for i in range(34):
    partData = all_data[:,i:i+1,:,:]
    norm_part = (partData - np.mean(partData))/(np.std(partData,ddof=10))
    norm_data.append(norm_part)
    print(norm_part.shape)
norm = np.concatenate(norm_data, axis=1)
print(norm.shape)

(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 1, 182, 232)
(2952, 34, 182, 232)


In [10]:
data_list = [norm[:744],norm[744:744+720],norm[744+720:744+744+720],norm[744+744+720:]]
for d in data_list: print(d.shape)
count = 0
for filename in sorted(glob.glob('/AS_data/METCRO2D_npy/METCRO2D_2015*')):
    with open(filename.replace('/AS_data/METCRO2D_npy/','/AS_data/METCRO2D_npy/NORM_'),'wb') as f:
        np.save(f,data_list[count])
    count += 1

(744, 34, 182, 232)
(720, 34, 182, 232)
(744, 34, 182, 232)
(744, 34, 182, 232)


In [23]:
overall_max = np.max(all_data)
overall_min = np.min(all_data)
overall_mean = np.mean(all_data)
overall_std = np.std(all_data, ddof = 10)

In [24]:
print(overall_max)
print(overall_min)
print(overall_mean)
print(overall_std)

5431.114
1e-27
30.106583
68.560036


In [25]:
# Method 1
norm_data = (all_data - overall_min)/(overall_max - overall_min)
data_list = [norm_data[:744],norm_data[744:744+720],norm_data[744+720:744+744+720],norm_data[744+744+720:]]
for d in data_list: print(d.shape)
count = 0
for filename in sorted(glob.glob('/AS_data/Conc_npy/NORM_*')):
    with open(filename,'wb') as f:
        np.save(f,data_list[count])
    count += 1

(744, 6, 182, 232)
(720, 6, 182, 232)
(744, 6, 182, 232)
(744, 6, 182, 232)


In [28]:
# Method 2
new_data = (all_data - overall_mean)/overall_std
norm_data = new_data - np.min(new_data)
data_list = [norm_data[:744],norm_data[744:744+720],norm_data[744+720:744+744+720],norm_data[744+744+720:]]
for d in data_list: print(d.shape)
count = 0
for filename in sorted(glob.glob('/AS_data/Conc_npy/ALL_NORM_*')):
    with open(filename,'wb') as f:
        np.save(f,data_list[count])
    count += 1

(744, 6, 182, 232)
(720, 6, 182, 232)
(744, 6, 182, 232)
(744, 6, 182, 232)


In [31]:
import json
with open('Norm1.json', "w", encoding='utf-8') as f:
    json.dump({'min':float(overall_min), 'max':float(overall_max)},f)
with open('Norm2.json', 'w', encoding='utf-8') as f:
    json.dump({'mean':float(overall_mean), 'std':float(overall_std), 'min':float(np.min(new_data))},f)

In [34]:
with open('Norm2.json', "r") as f:
    dic = json.load(f)
print(dic)

{'mean': 30.106582641601562, 'std': 68.5600357055664, 'min': -0.4391272962093353}


In [9]:
for filename in sorted(glob.glob('/AS_data/Conc_npy/TOTAL_2015_*')):
    print(filename)
    data = np.load(filename)
    print(data.shape)
    norm_data = (data - np.min(data))/(np.max(data) - np.min(data))
    print(norm_data)
    with open(filename.replace('/AS_data/Conc_npy/','/AS_data/Conc_npy/NORM_'), 'wb') as f:
        np.save(f,norm_data)

/AS_data/Conc_npy/TOTAL_2015_01_NO2_SO2_O3_PM25_PM10_CO__744_6_182_232.npy
(744, 6, 182, 232)
[[[[2.99408821e-05 1.85320150e-05 1.15342300e-05 ... 1.72843568e-06
    1.83621228e-06 1.91486038e-06]
   [4.08665001e-05 3.71422357e-05 2.98025534e-05 ... 1.80392044e-06
    1.93423875e-06 1.98657608e-06]
   [4.27108898e-05 3.92805996e-05 3.36302910e-05 ... 1.84967030e-06
    1.93343681e-06 1.98083967e-06]
   ...
   [4.27124905e-05 4.20582546e-05 4.07962143e-05 ... 1.52243756e-05
    1.38877722e-05 1.40622051e-05]
   [4.26901206e-05 4.30636392e-05 4.18097625e-05 ... 1.45636886e-05
    1.40450557e-05 1.35829596e-05]
   [4.26080624e-05 4.15290815e-05 3.98911179e-05 ... 1.54229692e-05
    1.48948075e-05 1.43054986e-05]]

  [[3.07276350e-05 1.30431845e-05 6.65780908e-06 ... 1.61459936e-20
    1.60319607e-20 1.59424151e-20]
   [3.93962109e-05 3.46372981e-05 2.98765171e-05 ... 1.61893835e-20
    1.61782472e-20 2.15365555e-24]
   [4.34715621e-05 3.86482279e-05 3.59174119e-05 ... 1.62288119e-20
    2

/AS_data/Conc_npy/TOTAL_2015_04_NO2_SO2_O3_PM25_PM10_CO__720_6_182_232.npy
(720, 6, 182, 232)
[[[[3.95144207e-06 3.42655289e-06 3.44175055e-06 ... 1.41209830e-06
    1.53635483e-06 1.83884652e-06]
   [3.66330255e-06 3.30699118e-06 3.13689179e-06 ... 1.45499121e-06
    1.49199570e-06 1.53291899e-06]
   [3.09819711e-06 2.91122888e-06 2.60260185e-06 ... 1.51166125e-06
    1.55584178e-06 1.57794659e-06]
   ...
   [1.30433637e-05 2.38247667e-05 2.93191642e-05 ... 2.15003674e-05
    2.08334532e-05 2.00146296e-05]
   [2.22071922e-05 3.45008666e-05 3.18970997e-05 ... 2.39762230e-05
    2.27220535e-05 2.10768940e-05]
   [1.85561730e-05 2.25044405e-05 2.05086344e-05 ... 2.74266458e-05
    2.63145012e-05 2.38817174e-05]]

  [[1.84999060e-14 1.84035350e-13 2.02608357e-13 ... 1.11336749e-17
    7.41868430e-18 3.89415126e-18]
   [1.18285237e-13 4.02636800e-13 1.78520837e-12 ... 1.12050868e-18
    1.19283994e-18 1.25219811e-18]
   [2.10265854e-12 7.67096543e-12 3.97127366e-11 ... 3.64981374e-19
    8

/AS_data/Conc_npy/TOTAL_2015_07_NO2_SO2_O3_PM25_PM10_CO__744_6_182_232.npy
(744, 6, 182, 232)
[[[[3.85355415e-05 1.64213143e-05 5.50714367e-06 ... 3.55549264e-06
    3.47296168e-06 3.27008865e-06]
   [5.72703429e-05 4.65026897e-05 2.24453779e-05 ... 3.24341772e-06
    3.08231756e-06 2.88852812e-06]
   [6.04989982e-05 5.88941803e-05 4.60237425e-05 ... 3.18607363e-06
    3.09114466e-06 2.85432952e-06]
   ...
   [1.74627221e-05 1.81985488e-05 2.31431695e-05 ... 1.27149281e-06
    1.52093946e-06 1.83451061e-06]
   [2.34234813e-05 3.19751707e-05 3.47712412e-05 ... 1.52261794e-06
    1.75918069e-06 1.95124335e-06]
   [3.22656815e-05 3.79642406e-05 4.28450403e-05 ... 1.62516289e-06
    1.75098592e-06 1.94185918e-06]]

  [[3.80965503e-05 7.69018970e-06 6.70796567e-07 ... 6.24466158e-24
    7.97558419e-21 2.00798659e-20]
   [5.64884103e-05 3.59709265e-05 1.30177250e-05 ... 7.48689016e-23
    8.87030361e-21 1.84885740e-20]
   [5.73565485e-05 4.95924323e-05 3.59184232e-05 ... 7.86681832e-22
    3

/AS_data/Conc_npy/TOTAL_2015_10_NO2_SO2_O3_PM25_PM10_CO__744_6_182_232.npy
(744, 6, 182, 232)
[[[[3.13193814e-05 9.27135807e-06 5.30563875e-06 ... 3.18771595e-06
    3.16864862e-06 3.08759013e-06]
   [4.92669569e-05 3.42407948e-05 1.52653647e-05 ... 2.40280315e-06
    2.41015459e-06 2.51862366e-06]
   [5.30547804e-05 4.75392553e-05 3.13369455e-05 ... 2.26192537e-06
    1.89158959e-06 2.13042972e-06]
   ...
   [1.40199836e-05 2.19083686e-05 2.49368422e-05 ... 2.05378346e-05
    1.99943461e-05 2.17208708e-05]
   [2.02766059e-05 3.44236396e-05 3.03584457e-05 ... 2.91543802e-05
    3.05954527e-05 3.09884308e-05]
   [1.90498013e-05 2.27865130e-05 2.22776198e-05 ... 3.60697595e-05
    3.55209013e-05 3.41604682e-05]]

  [[2.88058509e-05 4.08358528e-06 5.46539411e-07 ... 3.15121122e-23
    4.86927115e-24 1.06795329e-24]
   [4.96583707e-05 2.66709903e-05 8.52270750e-06 ... 1.95015051e-20
    1.96856530e-20 3.23000963e-21]
   [5.10423743e-05 3.82356666e-05 1.84395649e-05 ... 1.43038317e-20
    1

In [17]:
for filename in sorted(glob.glob('/AS_data/Conc_npy/TOTAL_2015_*')):
    print(filename)
    data = np.load(filename)
    print(data.shape)
    new_data = (data - np.mean(data))/np.std(data, ddof = 10)
    norm_data = new_data - np.min(new_data)
    with open(filename.replace('/AS_data/Conc_npy/','/AS_data/Conc_npy/ALL_NORM_'), 'wb') as f:
        np.save(f,norm_data)

/AS_data/Conc_npy/TOTAL_2015_01_NO2_SO2_O3_PM25_PM10_CO__744_6_182_232.npy
(744, 6, 182, 232)
/AS_data/Conc_npy/TOTAL_2015_04_NO2_SO2_O3_PM25_PM10_CO__720_6_182_232.npy
(720, 6, 182, 232)
/AS_data/Conc_npy/TOTAL_2015_07_NO2_SO2_O3_PM25_PM10_CO__744_6_182_232.npy
(744, 6, 182, 232)
/AS_data/Conc_npy/TOTAL_2015_10_NO2_SO2_O3_PM25_PM10_CO__744_6_182_232.npy
(744, 6, 182, 232)
